In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold
import torch.optim as optim
import random
import torchvision
from torch.nn import functional as F

In [2]:
import torch
from torchvision import datasets, transforms

# 假设数据集的路径
data_path = 'D:\code\classifier_data'

# 定义数据转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为 Tensor
    transforms.Normalize((0,), (255,))  # 归一化
])

# 加载 MNIST 数据集
mnist_dataset = datasets.MNIST(root=data_path, train=True, transform=transform, download=False)

# 将数据和标签分离
data = mnist_dataset.data
labels = mnist_dataset.targets

# 将数据和标签转换为 Tensor
data_all = data.unsqueeze(1)  # 添加通道维度 (N, C, H, W)，其中 C = 1
label_all = labels.long()  # 确保标签为 long 类型

# 打印数据和标签的形状
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)
print("Data :", data)




Data shape: torch.Size([60000, 28, 28])
Labels shape: torch.Size([60000])
Data : tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

 

In [3]:
class ConvNet(nn.Module):
    def __init__(self,h1=96):
        # We optimize dropout rate in a convolutional neural network.
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.drop1=nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(32 * 7 * 7, h1)
        self.drop2=nn.Dropout2d(p=0.1)
        self.fc2 = nn.Linear(h1, 10)
    def forward(self, x):
            x = F.relu(F.max_pool2d(self.conv1(x),kernel_size = 2))
            x = F.relu(F.max_pool2d(self.conv2(x),kernel_size = 2))
            x = self.drop1(x)
            x = x.view(x.size(0),-1)
            x = F.relu(self.fc1(x))
            x = self.drop2(x)
            x = self.fc2(x)
            return x 


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.CrossEntropyLoss()
num_epochs=10
batch_size=256
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={} 

In [5]:
def train_epoch(model,device,dataloader,loss_fn,optimizer):
  train_loss,train_correct=0.0,0
  model.train()
  for images, labels in dataloader:
    images,labels = images.to(device),labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = loss_fn(output,labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item() * images.size(0)
    scores, predictions = torch.max(output.data, 1)
    train_correct += (predictions == labels).sum().item()
  return train_loss,train_correct
def valid_epoch(model,device,dataloader,loss_fn):
        valid_loss, val_correct = 0.0, 0
        model.eval()
        for images, labels in dataloader:
            images,labels = images.to(device),labels.to(device)
            output = model(images)
            loss=loss_fn(output,labels)
            valid_loss+=loss.item()*images.size(0)
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()
        return valid_loss,val_correct 

In [6]:
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(data_all)))):
  #print(train_idx.shape,val_idx.shape)
  
  print('Fold {}'.format(fold + 1))
##############################
  n = len(train_idx)
  A = np.linspace(0, n - 1, n, dtype=int)
  random.shuffle(A)
  epoch_train = data_all[train_idx[A[:int(0.9 * n)]]]
  epoch_val = data_all[train_idx[A[int(0.9 * n):]]]
  epoch_test = data_all[val_idx]
  label_train = label_all[train_idx[A[:int(0.9 * n)]]]
  label_val = label_all[train_idx[A[int(0.9 * n):]]]
  label_test = label_all[val_idx]

    # 转换为Tensor并创建DataLoader
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#############################
  #train_sampler = SubsetRandomSampler(train_idx)
  #test_sampler = SubsetRandomSampler(val_idx)
  
  #train_loader = DataLoader(data_all, batch_size=batch_size, sampler=train_sampler)

  #test_loader = DataLoader(data_all, batch_size=batch_size, sampler=test_sampler)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = ConvNet()
  model.to(device)
  optimizer = optim.Adam(model.parameters(), lr=0.002)
  history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

 

  for epoch in range(num_epochs):
      train_loss, train_correct=train_epoch(model,device,train_loader,criterion,optimizer)
      print(f'train_loss type: {type(train_loss)}, train_correct type: {type(train_correct)}')
      test_loss, test_correct=valid_epoch(model,device,test_loader,criterion)
      train_loss = train_loss / len(train_loader.sampler)
      train_acc = train_correct / len(train_loader.sampler) * 100
      test_loss = test_loss / len(test_loader.sampler)
      test_acc = test_correct / len(test_loader.sampler) *100
      print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                                              num_epochs,
                                                                                                                              train_loss,
                                                                                                                              test_loss,
                                                                                                                              train_acc,
                                                                                                                              test_acc))
      history['train_loss'].append(train_loss)
      history['test_loss'].append(test_loss)
      history['train_acc'].append(train_acc)
      history['test_acc'].append(test_acc)
      foldperf['fold{}'.format(fold+1)] = history
      torch.save(model,'k_cross_CNN.pt')

Fold 1


C:\Users\FJQ\AppData\Local\Temp\ipykernel_32880\889207555.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_32880\889207555.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
d:\anaconda\envs\DL\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this

train_loss type: <class 'float'>, train_correct type: <class 'int'>
Epoch:1/10 AVG Training Loss:0.829 AVG Test Loss:0.131 AVG Training Acc 78.58 % AVG Test Acc 95.98 %
train_loss type: <class 'float'>, train_correct type: <class 'int'>
Epoch:2/10 AVG Training Loss:0.180 AVG Test Loss:0.075 AVG Training Acc 94.43 % AVG Test Acc 97.80 %
train_loss type: <class 'float'>, train_correct type: <class 'int'>
Epoch:3/10 AVG Training Loss:0.137 AVG Test Loss:0.064 AVG Training Acc 95.84 % AVG Test Acc 97.98 %
train_loss type: <class 'float'>, train_correct type: <class 'int'>
Epoch:4/10 AVG Training Loss:0.112 AVG Test Loss:0.054 AVG Training Acc 96.60 % AVG Test Acc 98.24 %
train_loss type: <class 'float'>, train_correct type: <class 'int'>
Epoch:5/10 AVG Training Loss:0.096 AVG Test Loss:0.053 AVG Training Acc 97.11 % AVG Test Acc 98.35 %
train_loss type: <class 'float'>, train_correct type: <class 'int'>
Epoch:6/10 AVG Training Loss:0.089 AVG Test Loss:0.053 AVG Training Acc 97.29 % AVG Tes

KeyboardInterrupt: 

In [7]:
testl_f,tl_f,testa_f,ta_f=[],[],[],[]
k=10
for f in range(1,k+1):
  tl_f.append(np.mean(foldperf['fold{}'.format(f)]['train_loss']))
  testl_f.append(np.mean(foldperf['fold{}'.format(f)]['test_loss']))
  ta_f.append(np.mean(foldperf['fold{}'.format(f)]['train_acc']))
  testa_f.append(np.mean(foldperf['fold{}'.format(f)]['test_acc']))
  print('Performance of {} fold cross validation'.format(k))
  print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc:{:.2f}".format(np.mean(tl_f),np.mean(testl_f),np.mean(ta_f),np.mean(testa_f)))

Performance of 10 fold cross validation
Average Training Loss: 0.180 	 Average Test Loss: 0.058 	 Average Training Acc: 94.95 	 Average Test Acc:98.08
Performance of 10 fold cross validation
Average Training Loss: 0.164 	 Average Test Loss: 0.060 	 Average Training Acc: 95.48 	 Average Test Acc:98.13
Performance of 10 fold cross validation
Average Training Loss: 0.159 	 Average Test Loss: 0.061 	 Average Training Acc: 95.59 	 Average Test Acc:98.12
Performance of 10 fold cross validation
Average Training Loss: 0.170 	 Average Test Loss: 0.063 	 Average Training Acc: 95.29 	 Average Test Acc:98.09
Performance of 10 fold cross validation
Average Training Loss: 0.164 	 Average Test Loss: 0.063 	 Average Training Acc: 95.41 	 Average Test Acc:98.13
Performance of 10 fold cross validation
Average Training Loss: 0.173 	 Average Test Loss: 0.063 	 Average Training Acc: 95.15 	 Average Test Acc:98.11
Performance of 10 fold cross validation
Average Training Loss: 0.176 	 Average Test Loss: 0.06